In [37]:
import csv
import requests
import xml.etree.ElementTree as ET
#import stanford_parser
import nltk
#from stanford_parser import sentence_split

In [38]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [132]:
# create element tree object 
def parse_file(xmlfiles,yrs):
   
    # open a file for writing

    QA_data = open('./data/qa4mre.csv', 'w')

    # create the csv writer object

    csvwriter = csv.writer(QA_data)

    # write column names
    headings = ['YR','T_ID','T_Name','R_ID','D_ID','D_STR','Q_ID','Q_STR','ANS_ID','ANS','CORRECT']
    csvwriter.writerow(headings)
    
    for i,xmlfile in enumerate(xmlfiles):
        print("Opening...",xmlfile)
        
        tree = ET.parse(xmlfile)
        yr = yrs[i]
  
        # get root element 
        root = tree.getroot() 

        #rows = []

        for topic in root.iter('topic'):
            t_id = topic.attrib['t_id']
            t_name = topic.attrib['t_name']
            #print(topic.attrib)

            for read_test in topic.iter('reading-test'):
                r_id = read_test.attrib['r_id']
                #print("\nreading test",r_id)
                doc = read_test.find('doc')
                d_id = doc.attrib['d_id']
                d_str = doc.text.encode('utf-8')

                for questions in read_test.iter('q'):
                    q_id = questions.attrib['q_id']
                    question = questions.find('q_str').text.encode('utf-8')
                    #print(question)

                    for answers in questions.iter('answer'):
                        rows = []

                        if ('correct' in answers.attrib):
                            correct = 1
                        else:
                            correct = 0
                        #print(correct)
                        a_id = answers.attrib['a_id']
                        a_text = answers.text.encode('utf-8')
                        #print(a_text)

                        rows.append(yr)
                        rows.append(t_id)
                        rows.append(t_name)
                        rows.append(r_id)                
                        rows.append(d_id)
                        rows.append(d_str)
                        rows.append(q_id)
                        rows.append(question)
                        rows.append(a_id)
                        rows.append(a_text)
                        rows.append(correct)

                        csvwriter.writerow(rows)

        print("Closing...",xmlfile)

    QA_data.close()

In [133]:
parse_file(files,years)

Opening... ./data/qa4mre/2011/QA4MRE-2011-EN_GS.xml
Closing... ./data/qa4mre/2011/QA4MRE-2011-EN_GS.xml
Opening... ./data/qa4mre/2012/QA4MRE-2012-EN_GS_SYNC.xml
Closing... ./data/qa4mre/2012/QA4MRE-2012-EN_GS_SYNC.xml
Opening... ./data/qa4mre/2012/QA4MRE-2012-EN_GS.xml
Closing... ./data/qa4mre/2012/QA4MRE-2012-EN_GS.xml
Opening... ./data/qa4mre/2013/QA4MRE-2013-EN_GS.xml
Closing... ./data/qa4mre/2013/QA4MRE-2013-EN_GS.xml


In [131]:
files = ['./data/qa4mre/2011/QA4MRE-2011-EN_GS.xml','./data/qa4mre/2012/QA4MRE-2012-EN_GS_SYNC.xml',
         './data/qa4mre/2012/QA4MRE-2012-EN_GS.xml','./data/qa4mre/2013/QA4MRE-2013-EN_GS.xml']
years = [2011,2012,2012,2013] # THIS NEEDS TO BE UPDATED AS FILES ARE ADDED/REMOVED

In [ ]:
# FILES TO SAVE FOR LATER
'./data/qa4mre/2012/QA4MRE-2012_BIOMEDICAL_GS.xml','./data/qa4mre/2013/QA4MRE-2013_BIO_GS-RUN.xml',